In [18]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
import os


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

model_rms = Model(inputs=base_model.input, outputs=x)
for layer in base_model.layers:
    layer.trainable = False

model_rms.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, )

train_generator = train_datagen.flow_from_directory(
    'E://MS_USD//Course 2_AAI-501//FinalProject//Train',  # Replace with the path to your train folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 12966 images belonging to 2 classes.


In [6]:
train_generator.class_indices

{'Accident': 0, 'NonAccident': 1}

In [7]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'E://MS_USD//Course 2_AAI-501//FinalProject//Validation',  # Replace with the path to your validation folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 4322 images belonging to 2 classes.


In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'E://MS_USD//Course 2_AAI-501//FinalProject//Test',  # Replace with the path to your validation folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 4323 images belonging to 2 classes.


In [19]:
early_stopping = EarlyStopping(
monitor='val_loss', # Monitor validation accuracy
# min_delta=0.001, # Minimum change to qualify as an improvement
patience=10, # Number of epochs to wait for an improvement
verbose=1,
# mode='max', # Stop when the quantity monitored has stopped increasing
restore_best_weights=True # Restore the weights of the best epoch
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    patience=5, 
    min_lr=0.00001
    )

In [21]:
def trainModel(model):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        callbacks=[early_stopping]
        )
    return history


In [23]:
trainedHistory_rms = trainModel(model_rms)

Epoch 1/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 2705s 7s/step - accuracy: 0.7704 - loss: 0.5302 - val_accuracy: 0.3558 - val_loss: 2.4718
Epoch 2/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7188 - loss: 0.5483 - val_accuracy: 0.5000 - val_loss: 0.8521
Epoch 3/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 2755s 7s/step - accuracy: 0.7895 - loss: 0.4410 - val_accuracy: 0.3637 - val_loss: 2.7428
Epoch 4/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.7188 - loss: 0.3738 - val_accuracy: 0.5000 - val_loss: 0.9395
Epoch 5/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 2739s 7s/step - accuracy: 0.7829 - loss: 0.4327 - val_accuracy: 0.3475 - val_loss: 3.7533
Restoring model weights from the end of the best epoch: 2.


In [24]:
import pickle 
with open('E:\\MS_USD\\Course 2_AAI-501\\FinalProject\\RmsPropModel\\model_30Epochs_history.pkl', 'wb') as file: 
    pickle.dump(trainedHistory_rms.history,file)

In [25]:
model_rms.save('E:\\MS_USD\\Course 2_AAI-501\\FinalProject\\RmsPropModel\\rms_model_30Epochs.keras')